In [1]:
import os
import sys
from time import time
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import networkx as nx
import random

from keras import layers, models, optimizers
from keras import backend as K
K.set_image_data_format('channels_last')
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks


from utils import combine_images
from utils import plot_log

from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
sys.path.append('./PatchyTools/')
from PatchyConverter import PatchyConverter
from DropboxLoader import DropboxLoader
from CapsuleParameters import CapsuleParameters
from CapsuleParameters import CapsuleTrainingParameters
from GraphClassifier import GraphClassifier


import matplotlib.pyplot as plt
%matplotlib inline

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Load data

In [35]:
# Getting the data:
dataset_name = 'ENZYMES'
width = 18
receptive_field = 10
relabeling = True

In [36]:
# Adjeceny dict and 
data_graph = PatchyConverter(dataset_name, receptive_field)
# Converting Graphs into Matrices:
if relabeling:
    data_graph.relabel_graphs()

importing graph data
number of graphs in ENZYMES dataset : 600
number of features : 3
Separating Graphs per graph ID
getting width
width: 33


In [37]:
graph_tensor = data_graph.graphs_to_Patchy_tensor()

ENZYMES tensor exists, loading it from Dropbox
Loading path: /Users/marcelogutierrez/.gamma_link/Samples/ENZYMES/ENZYMES_patchy_tensor_w_33_relabelled.npy


In [38]:
dropbox_loader = DropboxLoader(dataset_name)
graph_labels = dropbox_loader.get_graph_label()
graph_labels = np.array(graph_labels.graph_label)

In [57]:
# Capsule Architecture Parameters:
capsule_params= CapsuleParameters()

# First conv layer: 'filters', kernel_size)
conv_layer_name = 'conv_layer'
conv_layer_params = {}
conv_layer_params['filters'] = 256
conv_layer_params['kernel_size'] = 9
conv_layer_params['strides'] = [1, 1]
conv_layer_params['padding'] = 'VALID'
conv_layer_params['activation'] = 'relu'
conv_layer_params['name'] = 'conv1'

capsule_params.add_params(conv_layer_params,conv_layer_name)

# First Capsule Layer:
# [num_output_caps, caps_len,'filters',kernel_size,strides,padding]
caps_layer_name = 'caps_layer'
caps_layer_params = {}
caps_layer_params['filters'] = 256
caps_layer_params['kernel_size'] = 2
caps_layer_params['strides'] = [2,2]
caps_layer_params['padding'] = 'VALID'
caps_layer_params['padding'] = 'VALID'
caps_layer_params['n_channels'] = 32
caps_layer_params['dim_capsule'] = 8
caps_layer_params['name'] = 'caps_layer'
capsule_params.add_params(caps_layer_params,caps_layer_name)

# Digit Capsule Layer:
digit_layer_name = 'digitcaps_layer'
digit_layer_params = {}
digit_layer_params['n_channels'] = 10
digit_layer_params['dim_capsule'] = 16
digit_layer_params['name'] = 'digitcaps'
capsule_params.add_params(digit_layer_params,digit_layer_name )

# Capsule Decoder:
decoder_layer = 'decoder_layer'
decoder_params = {}
decoder_params['first_dense'] = 256 #250 #512
decoder_params['second_dense'] = 512
decoder_params['name'] = 'decoder'
capsule_params.add_params(decoder_params,decoder_layer)


In [58]:
parameter_set = CapsuleTrainingParameters()
parameter_set.add_fold(1)
parameter_set.epochs= 5

In [59]:
x_train, x_test, y_train, y_test = train_test_split(graph_tensor,graph_labels,test_size=0.10,random_state=1)
data = ((x_train, y_train), (x_test, y_test))
input_shape = x_train.shape[1:]

In [73]:
patchy_classifier = GraphClassifier(input_shape,6)
patchy_classifier.build_the_graph(capsule_params)
patchy_classifier.train(data, parameter_set)

time to generate the model: 0.26338791847229004
Train on 540 samples, validate on 60 samples
Epoch 1/5
150/540 [=======>......................] - ETA: 2s - loss: 0.9196 - capsnet_loss: 0.8092 - decoder_loss: 0.2816 - capsnet_acc: 0.1600

KeyboardInterrupt: 

In [72]:
patchy_classifier.y_train.shape

(540, 6)

In [63]:
patchy_classifier.train_model.input_shape

[(None, 33, 10, 3), (None, 2)]

In [34]:
patchy_classifier.train_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 18, 10, 7)    0                                            
__________________________________________________________________________________________________
relu (Conv2D)                   (None, 10, 2, 256)   145408      input_4[0][0]                    
__________________________________________________________________________________________________
primarycap_conv2d (Conv2D)      (None, 5, 1, 256)    262400      relu[0][0]                       
__________________________________________________________________________________________________
primarycap_reshape (Reshape)    (None, 160, 8)       0           primarycap_conv2d[0][0]          
__________________________________________________________________________________________________
primarycap

In [23]:
patchy_classifier.train_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 33, 10, 3)    0                                            
__________________________________________________________________________________________________
relu (Conv2D)                   (None, 25, 2, 256)   62464       input_1[0][0]                    
__________________________________________________________________________________________________
primarycap_conv2d (Conv2D)      (None, 12, 1, 256)   262400      relu[0][0]                       
__________________________________________________________________________________________________
primarycap_reshape (Reshape)    (None, 384, 8)       0           primarycap_conv2d[0][0]          
__________________________________________________________________________________________________
primarycap

In [25]:
patchy_classifier.m

AttributeError: 'dict' object has no attribute 'shape'

In [ ]:
patchy_classifier.results.rename('wegewg',inplace=True)

In [ ]:
patchy_classifier.results.name